*•* wspólne metody słowników
• specjalna obsługa brakujących kluczy
• odmiany typu dict w bibliotece standardowej
• typy set i frozenset
• działanie tablic mieszających
• konsekwencje stosowania tablic mieszających (ograniczenia typu kluczy, nieprzewidywalne
uporządkowanie itp.)

In [ ]:
import collections.abc as abc

my_dict = {"imie":"Tomek"}
isinstance(my_dict, abc.Mapping)

True

In [ ]:
ID = [
        (1, 'Tomek'),
        (2,'Janek'),
        (3,'Franek'),
        (4,'Matuesz'),
        (5,'Adam'),
        (6,'Ania'),
        (7,'Dagmara'),
        (8,'Robert'),
        (9,'Wojtek'),
]

lista_obecnosci = {id: imie for id,imie in ID}
lista_obecnosci

{1: 'Tomek',
 2: 'Janek',
 3: 'Franek',
 4: 'Matuesz',
 5: 'Adam',
 6: 'Ania',
 7: 'Dagmara',
 8: 'Robert',
 9: 'Wojtek'}

In [ ]:
import sys
import re 
WORD_RE = re.compile('\w+')
index = {}
with open(sys.argv[1],encoding='utf-8') as fp:
    for line_no, line in enumerate(fp,1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            occurences = index.get(word,[])
            occurences.append(location)
            index[word]=occurences
for word in sorted(index, key=str.upper):
    print(word, index[word])
            

In [ ]:
class StrKeyDict0(dict):  # <1>

    def __missing__(self, key):
        if isinstance(key, str):  # <2>
            raise KeyError(key)
        return self[str(key)]  # <3>

    def get(self, key, default=None):
        try:
            return self[key]  # <4>
        except KeyError:
            return default  # <5>

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()  # <6>


d = StrKeyDict0([('2','dwa'),('4','cztery')])
d.get(1,'Brak klucza')
1 in d


False

In [ ]:
import collections

class StrKeyDict1(collections.UserDict):  # <1>

    def __missing__(self, key):
        if isinstance(key, str):  # <2>
            raise KeyError(key)
        return self[str(key)]  # <3>

    def __contains__(self, key):
        return key in self.data
    
    def __setitem__(self,key,item):
        self.data[str(key)] = item
    
    

In [ ]:
import collections
from collections import ChainMap
import builtins
look = ChainMap(locals(),globals(),vars(builtins))
look2 = collections.Counter('asdffffghjkl')
look2.update("aaaaazzzzz")
look2.most_common(3)
look

In [ ]:
from types import MappingProxyType
d = {1:'a',2:'b'}
d_proxy = MappingProxyType(d)
d_proxy[2] = 'x' # tak się nie da!


In [ ]:
z = ['spam','tomek','spam','tomek']
print(set(z))
list(set(z))

# found = len(email1 & email2)
# found = len(set(email1).intersection(email2))

# found = 0
# for n in email1:
#     if n in email2:
#         found += 1

{'spam', 'tomek'}


['spam', 'tomek']

In [ ]:
from dis import dis
s = {1}
type(s)
s.pop()
s
frozenset(range(10))
dis('{1}')
dis('set([1])')
dis('frozenset(range(10))')

In [ ]:
from unicodedata import name
{chr(i) for i in range(32,256) if 'SIGN' in name(chr(i),'')}

# 1000 1x 0,000202s
# 10000 10x 0,000140s
#...
# 10 000 000 10 000x 0,000337s

• Znaki, punkty kodowe i reprezentacje bajtowe
• Unikalne cechy sekwencji binarnych: bytes, bytearray i memoryview
• Kodeki do pełnego kodowania Unicode i starszych zbiorów znaków
• Unikanie błędów kodowania i radzenie sobie z nimi
• Najlepsze praktyki podczas obsługi plików tekstowych
• Pułapka domyślnego kodowania i problemy dotyczące standardowych operacji I/O
• Bezpieczne porównania tekstu Unicode dzięki normalizacji
• Funkcje narzędziowe do normalizacji, sprowadzania do jednego rejestru i siłowego
usuwania znaków diakrytycznych
• Właściwe sortowanie tekstów Unicode przy użyciu modułu locale i biblioteki
PyUCA
• Metadane znaków w bazie danych Unicode
• Dwutrybowe interfejsy API, które obsługują typy str i bytes

In [ ]:
#print("U+0041") # odpowiednik \x41 w bajtach

kawa = bytes('café',encoding='utf-8')

kawa[0]
kawa[:1]

kawa_arr= bytearray(kawa)
kawa_arr[-1:]


# s = 'café'
# len(s)
# b = s.encode('utf-8')
# b
# len(b)
# b.decode('utf-8')

bytearray(b'\xa9')

In [ ]:
import array

number = array.array('h',[-2,-1,0,1,2])
dane = bytes(number)
dane

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

In [ ]:
import struct 
fmt = '<3s3sHH'
with open('filter.gif','rb') as fp:
    img = memoryview(fp.read())

header = img[:10]
bytes(header)
struct.unpack(fmt,header)
del header
del img

In [ ]:
for codec in ['latin_1','utf-8','utf-16']:
    print(codec, 'El Niño'.encode(codec),sep='\t')


latin_1	b'El Ni\xf1o'
utf-8	b'El Ni\xc3\xb1o'
utf-16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


In [ ]:
miasto = 'São Paulo'
miasto.encode('utf-8')
miasto.encode('utf_16')
miasto.encode('iso8859_1')
miasto.encode('cp437', errors='ignore')
miasto.encode('cp437', errors='replace')
miasto.encode('cp437', errors='xmlcharrefreplace')


b'S&#227;o Paulo'

In [ ]:
dane = b'Montr\xe9al'
dane.decode('cp1252')
dane.decode('iso8859_7')
dane.decode('koi8_r')
dane.decode('utf-8', errors='replace')

'Montr�al'

In [ ]:
#open('kawa.txt','w',encoding='utf-8').write('café')
open('kawa.txt').read()

'café'

Normalizacje Unicode w celu rozsądniejszego porównywania

In [ ]:
from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301'
len(s1),len(s2)
s1 == s2
len(normalize('NFC',s1)),len(normalize('NFC',s2))
normalize('NFC',s1) == normalize('NFC',s2)
len(normalize('NFD',s1)),len(normalize('NFD',s2))
# print(normalize('NFC',s1))
# print(normalize('NFC',s2))

café
café


In [ ]:
from unicodedata import normalize,name

ohm = '\u2126'
name(ohm)
ohm_c = normalize('NFC',ohm)
ohm_c
ohm == ohm_c
normalize('NFC',ohm) == normalize('NFC',ohm_c)

True

In [ ]:
from unicodedata import normalize,name
polowa = '½'
normalize('NFKC',polowa)

'1⁄2'

In [ ]:
import locale

locale.setlocale(locale.LC_COLLATE,'pt_BR.UTF-8')

fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']

sorted_fruits = sorted(fruits,key=locale.strxfrm)

print(sorted_fruits)


In [ ]:
def factorial(n):
    '''zwraca n!'''
    return 1 if n < 2 else n * factorial(n-1)


dir(factorial)

# factorial(42)
# factorial.__doc__
# type(factorial)
# help(factorial)
# fact = factorial
# fact(5)
#list(map(fact,range(11)))
#[fact(n) for n in range(11)]
#list(map(factorial,filter(lambda n: n% 2,range(11))))
# [factorial(n) for n in range(11) if n % 2]


['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
# from functools import reduce
# from operator import add

# reduce(add,range(100))

sum(range(100))


4950

In [ ]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits, key=lambda word: word[::-1])


['atemoia', 'acerola', 'caju', 'cajá', 'açaí']

In [ ]:
import random

class Lotek:
    
    def __init__(self,items):
        self._items = list(items)
        random.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('Pusta lista!!!')
    
    def __call__(self):
        return self.pick()


Dlotek = Lotek(range(1,49))
#Dlotek.pick()
Dlotek()
callable(Dlotek)

True

In [ ]:
 class C:
    pass


obj = C()

def f():
    pass

sorted(set(dir(f)) - set(dir(obj)))


['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

In [ ]:
def tag(name,*content,cls=None ,**attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr,value)
                            for attr,value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>'%(name,attr_str,c,name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)


tag('p',"witaj","World", cls='sidebar')
obraz = {'name':'img','title':'Kosmonauta','src':'filter.gif','cls':'framed'}
tag(**obraz)


def f(a,*,b):
    return a,b


f(1,b=2)

(1, 2)